# Distilled model of NLLB-200 with 600M parameters

In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import pydantic

model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

## (1) English to X

In [5]:
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M",src_lang="eng_Latn",tgt_lang="mar_Deva")

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

In [1]:
#English Dataset
with open('/kaggle/input/benchmarks689/benchmarks/wat2021-devtest/test.en', 'r') as file:
    en_dataset = file.readlines()

print(len(en_dataset))
max_length_en = max(len(line.strip()) for line in en_dataset)

2390


In [6]:
import random
num_lines = len(en_dataset)
indices = random.sample(range(num_lines), 1000)

In [5]:
print(indices)

NameError: name 'indices' is not defined

In [7]:
with open('/kaggle/working/indices.txt', 'w', encoding='utf-8') as file:
        for index in indices:
            file.write(str(index)+'\n')

### IF CRASHED

In [2]:
# List to store the integers
indices = []

# Read integers from the text file
with open('/kaggle/input/a3-1a-indices/indices.txt', 'r') as file:
    for line in file:
        indices.append(int(line.strip()))  # Convert each line to an integer and add it to the list


In [3]:
print(len(indices))

1000


In [4]:
sampled_en = []
for idx in indices:
    sampled_en.append(en_dataset[idx])

In [5]:
s_en = []
for sentence in sampled_en:
    words = sentence.split()
    s_en.append(words)

In [6]:
len(s_en)

1000

In [7]:
#Marathi Dataset
with open('/kaggle/input/benchmarks689/benchmarks/wat2021-devtest/test.mr', 'r') as file:
    mr_dataset = file.readlines()
#print(mr_dataset)
max_length_mr = max(len(line.strip()) for line in mr_dataset)

In [8]:
sampled_mr = []
for idx in indices:
    sampled_mr.append(mr_dataset[idx])

In [9]:
s_mr = []
for sentence in sampled_mr:
    words = sentence.split()
    s_mr.append(words)

In [10]:
predictions_en_mr = []
for i in range(0,1000,100):
    batch = tokenizer(sampled_en[i:i+100], return_tensors="pt", padding=True, truncation=True)
    translated_tokens = model.generate(**batch,forced_bos_token_id=tokenizer.lang_code_to_id["mar_Deva"],
        max_length=max_length_mr
    )
    outputs = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
    predictions_en_mr.extend(outputs)


the `lang_code_to_id` attribute is deprecated. The logic is natively handled in the `tokenizer.adder_tokens_decoder` this attribute will be removed in `transformers` v4.38


In [11]:
len(predictions_en_mr)

1000

In [12]:
with open('/kaggle/working/predictions_en_mr.txt', 'w', encoding='utf-8') as file:
        for sentence in predictions_en_mr:
            file.write(sentence.strip() + '\n')

## (2) X to English

In [13]:
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M",src_lang="mar_Deva",tgt_lang="eng_Latn")

In [14]:
predictions_mr_en = []
for i in range(0,1000,100):
    batch = tokenizer(sampled_mr[i:i+100], return_tensors="pt", padding=True, truncation=True)
    translated_tokens = model.generate(**batch,forced_bos_token_id=tokenizer.lang_code_to_id["eng_Latn"],
        max_length=max_length_en
    )
    outputs = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
    predictions_mr_en.extend(outputs)

In [15]:
len(predictions_mr_en)

1000

In [17]:
with open('/kaggle/working/predictions_mr_en.txt', 'w', encoding='utf-8') as file:
        for sentence in predictions_mr_en:
            file.write(sentence.strip() + '\n')

## (3) One Indian language (other than X), say Y, to X

In [10]:
#Hindi Dataset
with open('/kaggle/input/benchmarks689/benchmarks/wat2021-devtest/test.hi', 'r') as file:
    hi_dataset = file.readlines()
#print(hi_dataset)
max_length_hi = max(len(line.strip()) for line in hi_dataset)

In [11]:
sampled_hi = []
for idx in indices:
    sampled_hi.append(hi_dataset[idx])

In [12]:
s_hi = []
for sentence in sampled_hi:
    words = sentence.split()
    s_hi.append(words)

In [20]:
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M", src_lang="hin_Deva",tgt_lang="mar_Deva")

In [22]:
predictions_hi_mr = []
for i in range(0,1000,100):
    batch = tokenizer(sampled_hi[i:i+100], return_tensors="pt", padding=True, truncation=True)
    translated_tokens = model.generate(**batch,forced_bos_token_id=tokenizer.lang_code_to_id["mar_Deva"],
        max_length=max_length_mr
    )
    outputs = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
    predictions_hi_mr.extend(outputs)

In [23]:
len(predictions_hi_mr)

1000

In [25]:
with open('/kaggle/working/predictions_hi_mr.txt', 'w', encoding='utf-8') as file:
        for sentence in predictions_hi_mr:
            file.write(sentence.strip() + '\n')

## (4) X to Y

In [26]:
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M", src_lang="mar_Deva",tgt_lang="hin_Deva")

In [27]:
predictions_mr_hi = []
for i in range(0, 1000, 100):
    batch = tokenizer(sampled_mr[i:i+100], return_tensors="pt", padding=True, truncation=True)
    translated_tokens = model.generate(**batch,forced_bos_token_id=tokenizer.lang_code_to_id["hin_Deva"],
        max_length=max_length_hi
    )
    outputs = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
    predictions_mr_hi.extend(outputs)

In [28]:
len(predictions_mr_hi)

1000

In [29]:
with open('/kaggle/working/predictions_mr_hi.txt', 'w', encoding='utf-8') as file:
        for sentence in predictions_mr_hi:
            file.write(sentence.strip() + '\n')

## Q2 : BLEU and ROUGE scores(different variants)

### IF CRASHED

In [13]:
#EN-MR
predictions_en_mr = []
with open('/kaggle/input/ques1-a/predictions_en_mr.txt', 'r') as file:
    predictions_en_mr = file.readlines()

In [14]:
#MR-EN
predictions_mr_en = []
with open('/kaggle/input/ques1-a/predictions_mr_en.txt', 'r') as file:
    predictions_mr_en = file.readlines()

In [15]:
#HI-MR
predictions_hi_mr = []
with open('/kaggle/input/ques1-a/predictions_hi_mr.txt', 'r') as file:
    predictions_hi_mr = file.readlines()

In [16]:
#MR-HI
predictions_mr_hi = []
with open('/kaggle/input/ques1-a/predictions_mr_hi.txt', 'r') as file:
    predictions_mr_hi = file.readlines()

In [17]:
!pip install evaluate
!pip install rouge

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00


In [18]:
import evaluate
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu
import nltk

predictions = [predictions_en_mr, predictions_mr_en, predictions_hi_mr, predictions_mr_hi]
references = [s_mr,s_en,s_mr,s_hi]    #[sampled_mr, sampled_en, sampled_mr, sampled_hi]
tgt_lang = ['mar_Deva', 'eng_Latn', 'mar_Deva', 'hin_Deva']

bleu = evaluate.load("bleu")

for i in range(len(predictions)):
    print("Results for", tgt_lang[i], "are:\n")
    
    corpus_score = corpus_bleu(references[i], predictions[i])
    print("Corpus BLEU Score:", corpus_score, '\n')
    
    SentenceBleu = []
    
    for j in range(len(predictions[i])):
        hypothesis_tokens = nltk.word_tokenize(predictions[i][j].lower())
        reference_tokens = [nltk.word_tokenize(ref.lower()) for ref in references[i][j]]
        sentence_score = sentence_bleu(reference_tokens, hypothesis_tokens)
        SentenceBleu.append(sentence_score)
    
    AvgSentenceBleu = sum(SentenceBleu)/len(SentenceBleu)
    print("Average Sentence BLEU Score:", AvgSentenceBleu, '\n')

    results_bleu = bleu.compute(predictions=predictions[i], references=references[i])

    print(results_bleu,'\n')


2024-04-17 12:54:39.122647: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-17 12:54:39.122804: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-17 12:54:39.321257: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Results for mar_Deva are:

Corpus BLEU Score: 0.314813627415575 



/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Average Sentence BLEU Score: 0.6291820934924778 

{'bleu': 0.004896783832039565, 'precisions': [0.4373512226790738, 0.04236958718805877, 0.00033715441672285906, 9.203018590097552e-05], 'brevity_penalty': 1.0, 'length_ratio': 13.849150849150849, 'translation_length': 13863, 'reference_length': 1001} 

Results for eng_Latn are:

Corpus BLEU Score: 0.2980564088934616 

Average Sentence BLEU Score: 0.6660380432754996 

{'bleu': 0.009340529861294373, 'precisions': [0.5485089809916875, 0.0356724063445041, 0.0007893178977833323, 0.0004928536224741252], 'brevity_penalty': 1.0, 'length_ratio': 17.203, 'translation_length': 17203, 'reference_length': 1000} 

Results for mar_Deva are:

Corpus BLEU Score: 0.31890222061690987 

Average Sentence BLEU Score: 0.631046032549561 

{'bleu': 0.005165843828775641, 'precisions': [0.4458508141932731, 0.04263951619076782, 0.00041449059106358284, 9.03750564844103e-05], 'brevity_penalty': 1.0, 'length_ratio': 14.048951048951048, 'translation_length': 14063, 're

In [20]:
references = [sampled_mr, sampled_en, sampled_mr, sampled_hi]         
from rouge import Rouge
rouge = Rouge()

for i in range(len(predictions)):
    print("ROUGE Score for", tgt_lang[i], "are:")
    scores = rouge.get_scores(predictions[i], references[i],avg=True)
    print(scores,'\n')

ROUGE Score for mar_Deva are:
{'rouge-1': {'r': 0.42642577301124446, 'p': 0.44077743289078564, 'f': 0.42950448974611083}, 'rouge-2': {'r': 0.18272182212611773, 'p': 0.1872257425576459, 'f': 0.1833428305268514}, 'rouge-l': {'r': 0.40229766392962935, 'p': 0.41527219222940076, 'f': 0.4049366429718394}} 

ROUGE Score for eng_Latn are:
{'rouge-1': {'r': 0.5633854078391883, 'p': 0.5829999297378391, 'f': 0.5677754862018978}, 'rouge-2': {'r': 0.3011638448632576, 'p': 0.31049778390353755, 'f': 0.3027281078957012}, 'rouge-l': {'r': 0.5265151713489382, 'p': 0.5447179423326933, 'f': 0.5305549534953784}} 

ROUGE Score for mar_Deva are:
{'rouge-1': {'r': 0.4429642547650991, 'p': 0.4553505194489132, 'f': 0.44503532877616514}, 'rouge-2': {'r': 0.19273444420687239, 'p': 0.19647855515551912, 'f': 0.19287744480608307}, 'rouge-l': {'r': 0.41523752038560346, 'p': 0.426521785833567, 'f': 0.4169949718055695}} 

ROUGE Score for hin_Deva are:
{'rouge-1': {'r': 0.5177319682000064, 'p': 0.5467364506726362, 'f': 